# Genex Therapy Agents — Gross Motor Milestone → Next Steps → 5-Day Plan (ADK Notebook)

## What this notebook is
This notebook is a **Genex prototype “therapy agent pipeline”** built with **Google ADK (Agent Development Kit)**. It turns a parent’s free-text description of their child (profile + goals) into a structured, parent-friendly output:

1) **Stores the child profile** (name, age, diagnosis) in session state  
2) **Estimates developmental delay (in months)** using multiple LLMs (Gemini + GPT + Claude, when keys are available)  
3) **Builds CDC milestone yes/no questions** for *Movement & Physical (gross motor)* based on the child’s chronological age and estimated delay  
4) **Selects 3–6 “next-step” milestones** slightly above the estimated developmental age  
5) **Generates a 5-day at-home activity plan** aligned to those milestones (via GPT)  
6) **Summarizes** the reasoning and plan in a supportive tone (with a non-medical disclaimer)

> Important: This is a planning / educational prototype and **does not replace professional medical advice**.

---

## Inputs (what you must provide)
### 1) CDC milestone table (Excel)
The pipeline relies on an Excel file with CDC milestone rows and at least these columns:
- `category` (e.g., “movement and physical”)
- `milestone` (text description)
- `months` (numeric age in months)

**You must set:**
- `CDC_TABLE_PATH` to the location of your milestone Excel file.

### 2) API keys (environment variables)
This notebook reads keys from `.env` (via `python-dotenv`) and initializes clients for tool calls:
- `GOOGLE_API_KEY` (Gemini via `google.genai`)
- `OPENAI_API_KEY` (GPT via `openai`)
- `ANTHROPIC_API_KEY` (Claude via `anthropic`)

Notes:
- If a key is missing, the corresponding model/tool step may be skipped or return an error message depending on the function.

### 3) Parent prompts (runtime input)
You interact with the pipeline by calling:
- `await ask_gross_motor("...")`

Example first message typically includes:
- child name, age, diagnosis
- what the parent wants help with (e.g., gross motor + 5-day plan)

---

## Process overview (pipeline stages)
The notebook defines tools + agents and wires them into a **SequentialAgent**:

1. **Profile Agent**
   - Extracts name/age/diagnosis from parent text
   - Calls `save_child_profile()` to store in ADK session state

2. **Delay Estimator Tool (Multi-model)**
   - Calls Gemini/GPT/Claude (when available) to estimate “delay months”
   - Parses numeric month estimates and stores a combined result in session state

3. **Milestone Question Builder Tool**
   - Loads CDC milestones for the requested category (gross motor)
   - Uses chronological age and stored delay to compute an approximate developmental age window
   - Produces parent-facing yes/no/not sure questions and stores them as `pending_questions:*`

4. **Focus Milestones Tool**
   - Uses the computed developmental age to select 3–6 next milestones above that level
   - Stores them as `focus_milestones:*`

5. **Activity Plan Tool (GPT)**
   - Uses the focused milestones to generate a **5-day home activity plan**
   - Stores it as `activity_plan:*`

6. **Summary Agent**
   - Produces a friendly recap: estimated delay, dev age band, milestone targets, and how the plan supports them

---

## Outputs (what you get)
### Console output
- The `run_debug()` helper prints agent messages and tool call traces (useful for debugging).
- You’ll see:
  - clarifying questions (if profile is missing)
  - milestone question sets
  - selected milestones
  - the 5-day plan text

### Session state artifacts (in-memory)
This notebook uses `InMemorySessionService`, so state persists **only during the notebook runtime** for a given `user_id` + `session_id`. Common keys include:
- `child_profile` (name, age_years, diagnosis)
- `delay_months:*` or similar delay fields
- `dev_age:{category}:months`
- `pending_questions:{category}`
- `focus_milestones:{category}`
- `activity_plan:{category}`

---

## How to run
1) Run cells top-to-bottom to load keys, load the CDC table, and register tools/agents.  
2) Start a session by sending a profile + intent:
   - `await ask_gross_motor("My child's name is ..., she is ... years old, diagnosis ..., help with gross motor and a 5-day plan.")`
3) Continue the conversation with follow-ups:
   - simplify a day, make it gentler, adjust for constraints, etc.

---

In [ ]:
import openai
import anthropic
import google.genai
from google.adk import agents
print("All good!")

In [2]:
# ============================================
# 0. Imports & basic setup
# ============================================
import os
from typing import Any, Dict, List
import re
import statistics

import pandas as pd
from dotenv import load_dotenv

# ADK core imports
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.apps.app import App
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext

# LiteLlm wrapper (routes calls to OpenAI via litellm)
from google.adk.models.lite_llm import LiteLlm

# Direct API clients used *inside tools*
from openai import OpenAI
from anthropic import Anthropic
import google.genai as genai  # only for inside tools; errors are caught

# Ensure we use direct Gemini API, not Vertex
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

# Load API keys
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# Direct clients (used ONLY in tools, not in ADK models)
gemini_client = genai.Client(api_key=GOOGLE_API_KEY) if GOOGLE_API_KEY else None
openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY) if ANTHROPIC_API_KEY else None

print("API keys (first 5 chars):")
print("  Google:", (GOOGLE_API_KEY or "MISSING")[:5])
print("  OpenAI:", (OPENAI_API_KEY or "MISSING")[:5])
print("  Anthropic:", (ANTHROPIC_API_KEY or "MISSING")[:5])

# Session service for THIS notebook only
session_service = InMemorySessionService()

API keys (first 5 chars):
  Google: AIzaS
  OpenAI: sk-pr
  Anthropic: sk-an


In [ ]:
# ============================================
# 1. CDC milestone table loader + helpers
# ============================================

CDC_TABLE_PATH = r"C:/Users/T490/Downloads/Genetics-Dashboard/milestone-cdc-table.xlsx"

cdc_df = pd.read_excel(CDC_TABLE_PATH)
cdc_df.columns = cdc_df.columns.str.strip()

# Normalize columns
cdc_df["category_clean"]  = cdc_df["category"].astype(str).str.strip().str.lower()
cdc_df["milestone_clean"] = cdc_df["milestone"].astype(str).str.strip()
cdc_df["months"]          = pd.to_numeric(cdc_df["months"], errors="coerce")

CDC_AGES = sorted(cdc_df["months"].dropna().unique())

# Map friendly input → CDC categories
CATEGORY_MAP = {
    "gross_motor": "movement and physical",
    "gross motor": "movement and physical",
    "physical": "movement and physical",
    "physical & movement": "movement and physical",
    "movement_and_physical": "movement and physical",
    "movement and physical": "movement and physical",
    "social_emotional": "social and emotional",
    "social emotional": "social and emotional",
    "social & emotional": "social and emotional",
    "language_communication": "language and communication",
    "language communication": "language and communication",
    "speech": "language and communication",
    "cognitive": "cognitive",
}

print("CDC milestone table loaded.")
print("Columns:", list(cdc_df.columns))
print("Sample ages (months):", CDC_AGES[:10], "...")

CDC milestone table loaded.
Columns: ['months', 'category', 'milestone', 'category_clean', 'milestone_clean']
Sample ages (months): [np.int64(2), np.int64(4), np.int64(6), np.int64(9), np.int64(12), np.int64(15), np.int64(18), np.int64(24), np.int64(30), np.int64(36)] ...


In [4]:
# ============================================
# 2. Child profile tools (save / retrieve)
# ============================================

def save_child_profile(
    tool_context: ToolContext,
    name: str,
    age_years: int,
    diagnosis: str,
) -> Dict[str, Any]:
    """
    Store child's profile in ADK session state.
    """
    clean_name = name.strip()
    clean_diagnosis = diagnosis.strip()

    tool_context.state["user:child:name"] = clean_name
    tool_context.state["user:child:age_years"] = int(age_years)
    tool_context.state["user:child:diagnosis"] = clean_diagnosis

    return {
        "status": "success",
        "message": f"Stored child profile: {clean_name}, {age_years} years, diagnosis: {clean_diagnosis}.",
        "name": clean_name,
        "age_years": int(age_years),
        "diagnosis": clean_diagnosis,
    }


def retrieve_child_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Retrieve child's profile from ADK session state.
    """
    name = tool_context.state.get("user:child:name")
    age_years = tool_context.state.get("user:child:age_years")
    diagnosis = tool_context.state.get("user:child:diagnosis")

    if name is None or age_years is None or diagnosis is None:
        return {
            "status": "error",
            "message": "Child profile is missing or incomplete. Please provide name, age, and diagnosis.",
        }

    return {
        "status": "success",
        "name": name,
        "age_years": age_years,
        "diagnosis": diagnosis,
    }


save_child_profile_tool = FunctionTool(func=save_child_profile)
retrieve_child_profile_tool = FunctionTool(func=retrieve_child_profile)

print("Child profile tools initialized.")

Child profile tools initialized.


In [5]:
# ============================================
# 3. Multi-model delay estimator (Gemini + GPT + Claude)
# ============================================

def _extract_delay_months(text: str | None) -> int:
    """
    Extract the first integer from the model's response.
    If nothing is found or text is None, return 0.
    """
    if not text:
        return 0
    first_line = text.strip().splitlines()[0]
    m = re.search(r"\d+", first_line)
    if not m:
        return 0
    return int(m.group(0))


def estimate_delay_multi_model(
    tool_context: ToolContext,
    category: str,
) -> Dict[str, Any]:
    """
    Ask Gemini, GPT, and Claude for a typical developmental delay (in months)
    for this child's diagnosis and this category, then store the median.

    Stores:
        state["delay:{category_key}:months"] = <int>
    """
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {
            "status": "error",
            "message": "Child profile is missing. Please provide name, age, and diagnosis first.",
        }

    name = profile["name"]
    age_years = profile["age_years"]
    diagnosis = profile["diagnosis"]

    cat_norm = CATEGORY_MAP.get(category.lower(), category).lower()

    base_prompt = f"""
You are a pediatric developmental specialist.

Child:
- Name: {name}
- Age: {age_years} years
- Diagnosis: {diagnosis}

Domain: "{cat_norm}"

Question:
For children ages 0–5 with {diagnosis}, what is a **typical average developmental delay**
in this domain compared to neurotypical peers?

Respond in this exact format:
FIRST LINE: a single integer number of months (e.g. 12)
THEN: one short sentence explaining the reasoning.
"""

    # ---- Gemini (tool-only; errors caught) ----
    gemini_text = ""
    if gemini_client:
        try:
            g_resp = gemini_client.models.generate_content(
                model="gemini-2.5-flash-lite",
                contents=base_prompt,
            )
            gemini_text = g_resp.text if hasattr(g_resp, "text") else str(g_resp)
        except Exception as e:
            gemini_text = f"Error: {e}"

    # ---- OpenAI GPT ----
    gpt_text = ""
    if openai_client:
        try:
            oai = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": base_prompt}],
            )
            gpt_text = oai.choices[0].message.content
        except Exception as e:
            gpt_text = f"Error: {e}"

    # ---- Claude ----
    claude_text = ""
    if anthropic_client:
        try:
            c_resp = anthropic_client.messages.create(
                model="claude-3-5-haiku-20241022",
                max_tokens=300,
                messages=[{"role": "user", "content": base_prompt}],
            )
            text_blocks = [
                b.text for b in c_resp.content
                if getattr(b, "type", None) == "text"
            ]
            claude_text = "\n".join(text_blocks).strip()
        except Exception as e:
            claude_text = f"Error: {e}"

    # ---- Parse integers ----
    gemini_months = _extract_delay_months(gemini_text)
    gpt_months = _extract_delay_months(gpt_text)
    claude_months = _extract_delay_months(claude_text)

    valid_estimates = [m for m in [gemini_months, gpt_months, claude_months] if m > 0]

    if not valid_estimates:
        median_delay = 0
    else:
        median_delay = int(statistics.median(valid_estimates))

    key = cat_norm.replace(" ", "_")
    tool_context.state[f"delay:{key}:months"] = median_delay

    return {
        "status": "success",
        "category": cat_norm,
        "delay_months": median_delay,
        "per_model": {
            "gemini_months": gemini_months,
            "gpt_months": gpt_months,
            "claude_months": claude_months,
        },
        "raw_text": {
            "gemini": gemini_text,
            "gpt": gpt_text,
            "claude": claude_text,
        },
    }


delay_estimator_tool = FunctionTool(func=estimate_delay_multi_model)
print("Multi-model delay estimator tool initialized.")


Multi-model delay estimator tool initialized.


In [6]:
# ============================================
# 4. CDC milestone question builder
# ============================================

def build_milestone_questions(
    tool_context: ToolContext,
    category: str,
    window_months: int = 12,
) -> Dict[str, Any]:
    """
    Use child's chronological age + stored delay to decide which CDC
    milestone ages to question, and return a list of parent-facing questions.

    Also stores:
        state["dev_age:{category_key}:months"] = approx_dev_months
    """
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {"status": "error", "message": "Child profile missing."}

    age_years = profile["age_years"]
    chrono_months = int(age_years * 12)

    cat_norm = CATEGORY_MAP.get(category.lower(), category).lower()
    delay_key = cat_norm.replace(" ", "_")
    delay_months = tool_context.state.get(f"delay:{delay_key}:months", 0)

    # Approx developmental age
    approx_dev_months = max(0, chrono_months - delay_months)

    # Store for downstream tools
    tool_context.state[f"dev_age:{delay_key}:months"] = approx_dev_months

    # Choose a window around this estimate
    min_months = max(2, approx_dev_months - window_months // 2)
    max_months = min(60, approx_dev_months + window_months // 2)

    # Snap to CDC buckets
    relevant_ages = [m for m in CDC_AGES if min_months <= m <= max_months]
    if not relevant_ages:
        relevant_ages = [min(CDC_AGES), max(CDC_AGES)]

    subset = cdc_df[
        (cdc_df["category_clean"] == cat_norm) &
        (cdc_df["months"].isin(relevant_ages))
    ].copy()

    questions: List[Dict[str, Any]] = []
    for i, row in subset.iterrows():
        q_id = f"{cat_norm}_{row['months']}_{i}"
        questions.append({
            "id": q_id,
            "months": int(row["months"]),
            "question": (
                f"At around {int(row['months'])} months, does {profile['name']} "
                f"'{row['milestone_clean']}'? (yes/no/not sure)"
            ),
        })

    tool_context.state[f"pending_questions:{delay_key}"] = questions

    return {
        "status": "success",
        "approx_dev_months": approx_dev_months,
        "chrono_months": chrono_months,
        "delay_months": delay_months,
        "questions": questions,
    }


milestone_questions_tool = FunctionTool(func=build_milestone_questions)
print("Milestone question builder initialized.")

Milestone question builder initialized.


In [7]:
# ============================================
# 5. Focus milestones tool (3–6 “next steps”)
# ============================================

def select_next_milestones(
    tool_context: ToolContext,
    category: str,
    max_milestones: int = 6,
) -> Dict[str, Any]:
    """
    Choose 3–6 CDC milestones immediately above the child's developmental age
    for the given category.

    Expects:
        state["dev_age:{category_key}:months"] from build_milestone_questions()
    """
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {"status": "error", "message": "Child profile missing."}

    cat_norm = CATEGORY_MAP.get(category.lower(), category).lower()
    key = cat_norm.replace(" ", "_")

    dev_age_months = tool_context.state.get(f"dev_age:{key}:months", None)
    if dev_age_months is None:
        return {
            "status": "error",
            "message": f"No developmental age stored for category '{category}'.",
        }

    min_m = dev_age_months + 1
    max_m = dev_age_months + 12

    subset = cdc_df[
        (cdc_df["category_clean"] == cat_norm) &
        (cdc_df["months"] >= min_m) &
        (cdc_df["months"] <= max_m)
    ].sort_values("months")

    if subset.empty:
        return {
            "status": "error",
            "message": f"No next-step milestones found for '{category}'.",
            "milestones": [],
        }

    milestones: List[Dict[str, Any]] = []
    for i, row in subset.iterrows():
        milestones.append({
            "id": f"{cat_norm}_{row['months']}_{i}",
            "months": int(row["months"]),
            "milestone": row["milestone_clean"],
        })

    selected = milestones[:max_milestones]
    tool_context.state[f"focus_milestones:{key}"] = selected

    return {
        "status": "success",
        "dev_age": dev_age_months,
        "category": cat_norm,
        "selected_milestones": selected,
    }


focus_milestones_tool = FunctionTool(func=select_next_milestones)
print("Focus-milestones tool initialized.")


Focus-milestones tool initialized.


In [8]:
# ============================================
# 6. Activity planner tool (5-day plan via GPT)
# ============================================

def gpt_generate_activity_plan(
    tool_context: ToolContext,
    category: str,
) -> Dict[str, Any]:
    """
    Generate a 5-day activity plan targeting the selected milestones.

    Expects:
        state["focus_milestones:{category_key}"]
    Uses OpenAI GPT-4o-mini.
    """
    if openai_client is None:
        return {"status": "error", "message": "OpenAI client not configured."}

    cat_norm = CATEGORY_MAP.get(category.lower(), category).lower()
    key = cat_norm.replace(" ", "_")

    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {"status": "error", "message": "Child profile missing."}

    name = profile["name"]
    age = profile["age_years"]
    diag = profile["diagnosis"]

    milestones = tool_context.state.get(f"focus_milestones:{key}", None)
    if not milestones:
        return {
            "status": "error",
            "message": f"No selected milestones found for category '{category}'. "
                       f"Run the focus-milestones tool first.",
        }

    milestone_list = "\n".join(
        f"- ({m['months']} months) {m['milestone']}"
        for m in milestones
    )

    prompt = f"""
You are a pediatric therapist helping parents at home.

Child:
- Name: {name}
- Age: {age} years
- Diagnosis: {diag}
- Target category: {cat_norm}

Target “next step” milestones:
{milestone_list}

TASK:
Create a **5-day home activity plan**, with 1–2 activities per day,
directly targeting these milestones.

REQUIREMENTS:
- Use warm, parent-friendly language.
- Make each activity short, practical, play-based, and doable at home.
- Avoid medical jargon or naming therapy models.
- Do NOT mention that you are an AI or that this is a prompt.

FORMAT:
Return in this exact JSON-style structure (no extra commentary):

{{
  "day_1": ["activity 1", "activity 2 optional"],
  "day_2": ["..."],
  "day_3": ["..."],
  "day_4": ["..."],
  "day_5": ["..."]
}}
"""

    try:
        result = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        content = result.choices[0].message.content
    except Exception as e:
        return {"status": "error", "message": f"GPT error: {e}"}

    tool_context.state[f"activity_plan:{key}"] = content

    return {
        "status": "success",
        "category": cat_norm,
        "plan": content,
    }


activity_plan_tool = FunctionTool(func=gpt_generate_activity_plan)
print("Activity-plan tool initialized.")


Activity-plan tool initialized.


In [9]:
# ============================================
# 7. Agents for the gross-motor (movement & physical) pipeline
# ============================================

# 7.1 Profile agent
milestone_profile_agent_gross_motor = LlmAgent(
    name="profile_agent_gross_motor",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    description="Collects and stores child profile info (name, age, diagnosis) in session state for the milestone pipeline.",
    instruction="""
You are a profile manager for a pediatric support system.

1. When the parent describes their child (name, age, diagnosis),
   you MUST call `save_child_profile` to store those details.

2. If they update the info, call `save_child_profile` again.

3. After saving, briefly confirm what you stored, e.g.:
   "I've saved Emma's profile: 3 years old with Down syndrome."

Do NOT give therapy recommendations. Only manage the stored profile.
""",
    tools=[save_child_profile_tool, retrieve_child_profile_tool],
)

# 7.2 Delay agent
delay_agent_gross_motor = LlmAgent(
    name="delay_agent_gross_motor",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    tools=[delay_estimator_tool],
    description="Gets a multi-model vote on developmental delay (months) for gross motor.",
    instruction="""
You estimate developmental delay in **gross motor / movement and physical**.

Rules:
- ALWAYS call `estimate_delay_multi_model` exactly once.
- Always pass the category string "movement and physical".
- When the tool returns, briefly explain:
    * the median delay in months
    * the range across models (if they differ)
- Do NOT recommend activities; other agents handle that.
""",
)

# 7.3 Milestone Q&A agent
milestone_qna_agent_gross_motor = LlmAgent(
    name="milestone_qna_agent_gross_motor",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    tools=[milestone_questions_tool],
    description="Asks CDC-based milestone questions to estimate gross motor developmental age.",
    instruction="""
You help parents answer CDC milestone questions for gross motor skills
(the 'movement and physical' category).

1. ALWAYS call `build_milestone_questions` once with category "movement and physical".
2. Present those questions clearly numbered, and ask the parent to reply with
   yes/no/not sure for each.
3. Use `approx_dev_months` from the tool result to say something like:
   "Based on this, Emma's gross motor skills are roughly around the 18–24 month level."

Do NOT recommend specific activities; that is for later agents.
(For now you don't need to adjust the dev age based on detailed yes/no parsing.)
""",
)

# 7.4 Focus agent: 3–6 next milestones
focus_agent_gross_motor = LlmAgent(
    name="focus_agent_gross_motor",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    tools=[focus_milestones_tool],
    description="Chooses 3–6 next CDC milestones for gross motor based on stored developmental age.",
    instruction="""
You select “next-step” CDC milestones for gross motor.

1. ALWAYS call `select_next_milestones` once with category "movement and physical".
2. When the tool returns:
   - Briefly explain which age band the milestones are from (e.g., 18–24 months).
   - List the selected milestones in a parent-friendly way.

Do not invent milestones; rely entirely on the tool output.
""",
)

# 7.5 Activity agent: 5-day plan
activity_agent_gross_motor = LlmAgent(
    name="activity_agent_gross_motor",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    tools=[activity_plan_tool],
    description="Generates a 5-day home activity plan for the selected gross motor milestones.",
    instruction="""
You create a 5-day home activity plan for gross motor skills.

1. ALWAYS call `gpt_generate_activity_plan` once with category "movement and physical".
2. When the tool returns with JSON-style text, present it clearly to the parent:
   - Day 1: ...
   - Day 2: ...
   - etc.

Do NOT change the meaning of the plan, only format it more readably.
""",
)

# 7.6 Final summary agent
milestone_summary_agent_gross_motor = LlmAgent(
    name="milestone_summary_agent_gross_motor",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    tools=[retrieve_child_profile_tool],
    description="Summarizes the gross motor assessment and plan for parents.",
    instruction="""
You are the final summarizer for the gross motor pipeline.

1. Call `retrieve_child_profile` to get the child's name, age, and diagnosis.
2. Then write a short summary with this structure:

   "[NAME] is [AGE] years old and has [DIAGNOSIS]."

   Then 1–2 short paragraphs explaining:
     - the estimated delay in gross motor (if mentioned earlier),
     - the approximate developmental age band (e.g. "around 18–24 months"),
     - what the next-step milestones focus on,
     - and how the 5-day plan supports those milestones.

3. Optionally add a simple bullet list:
   - "This week you'll focus on: [milestone 1], [milestone 2], ..."

Use a warm, supportive tone and remind parents this does not replace
professional medical advice.
""",
)

print("All gross-motor agents initialized.")


All gross-motor agents initialized.


In [10]:
# ============================================
# 8. Gross motor pipeline, app, runner
# ============================================

gross_motor_pipeline = SequentialAgent(
    name="gross_motor_pipeline",
    sub_agents=[
        milestone_profile_agent_gross_motor,   # Step 0: capture/confirm profile
        delay_agent_gross_motor,               # Step 1: multi-model delay estimate
        milestone_qna_agent_gross_motor,       # Step 2: CDC questions + approx dev-age
        focus_agent_gross_motor,               # Step 3: 3–6 next milestones
        activity_agent_gross_motor,            # Step 4: 5-day plan
        milestone_summary_agent_gross_motor,   # Step 5: recap for parents
    ],
)

milestone_app = App(
    name="genex_milestones_app",
    root_agent=gross_motor_pipeline,
)

milestone_runner = Runner(
    app=milestone_app,
    session_service=session_service,
)

print("Milestone app + gross motor pipeline initialized.")

Milestone app + gross motor pipeline initialized.


In [11]:
# ============================================
# 9. Helper: talk to this pipeline
# ============================================

async def ask_gross_motor(prompt: str):
    """
    Send a message into the gross motor pipeline and print all debug events.
    """
    events = await milestone_runner.run_debug(
        prompt,
        user_id="emma_parents",
        session_id="emma-gross-001",
        quiet=False,   # show messages
        verbose=True,  # show tool calls, etc.
    )
    return events

print("ask_gross_motor helper ready.")

ask_gross_motor helper ready.


### Start a new session & give profile + intent

In [12]:
await ask_gross_motor(
    "My child's name is Emma. She is 3 years old and has Down syndrome. "
    "Please assess her gross motor skills and create a 5-day home activity plan."
)



 ### Created new session: emma-gross-001

User > My child's name is Emma. She is 3 years old and has Down syndrome. Please assess her gross motor skills and create a 5-day home activity plan.
profile_agent_gross_motor > [Calling tool: save_child_profile({'name': 'Emma', 'age_years': 3, 'diagnosis': 'Dow...)]
profile_agent_gross_motor > [Tool result: {'status': 'success', 'message': 'Stored child profile: Emma, 3 years, diagnosis: Down syndrome.', '...]
profile_agent_gross_motor > I've saved Emma's profile: 3 years old with Down syndrome.
delay_agent_gross_motor > [Calling tool: estimate_delay_multi_model({'category': 'movement and physical'})]
delay_agent_gross_motor > [Tool result: {'status': 'success', 'category': 'movement and physical', 'delay_months': 24, 'per_model': {'gemini...]
delay_agent_gross_motor > Emma's estimated developmental delay in gross motor skills is **24 months**, based on a multi-model assessment.

The individual model outputs vary: 
- **Gemini** indicated a si

[Event(model_version='gpt-4o-mini-2024-07-18', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'age_years': 3,
           'diagnosis': 'Down syndrome',
           'name': 'Emma'
         },
         id='call_5NG6oHfbfHUDAAZTkat1ZUH3',
         name='save_child_profile'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=False, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   cached_content_token_count=0,
   candidates_token_count=27,
   prompt_token_count=253,
   total_token_count=280
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-178eaf5d-6696-4025-8310-170b1d975d9f', author='profile_agent_gross_motor', actions=EventA

In [13]:
await ask_gross_motor(
    "Can you simplify Day 3 activities for busy weekdays?"
)


 ### Continue session: emma-gross-001

User > Can you simplify Day 3 activities for busy weekdays?
profile_agent_gross_motor > Sure! Here’s a simplified version of Day 3 activities for Emma, suitable for a busy weekday:

### Simplified Day 3 Activities:

1. **Walking Encouragement:**
   - Have Emma walk from one side of the room to the other while you cheer her on. You can do this quickly, just take a few minutes to encourage her as she walks.

2. **Drink Practice:**
   - Give Emma a cup without a lid and let her practice drinking water. Supervise her closely to enjoy the experience, and it’s perfectly fine for her to spill a little!

These activities are quick but effective in promoting her gross motor skills while fitting into a busy schedule. Enjoy your time together!
delay_agent_gross_motor > Here’s a simplified version of Day 3 activities for Emma, suitable for a busy weekday:

### Simplified Day 3 Activities:

1. **Walking Encouragement:**
   - Have Emma walk from one side of th

[Event(model_version='gpt-4o-mini-2024-07-18', content=Content(
   parts=[
     Part(
       text="""Sure! Here’s a simplified version of Day 3 activities for Emma, suitable for a busy weekday:
 
 ### Simplified Day 3 Activities:
 
 1. **Walking Encouragement:**
    - Have Emma walk from one side of the room to the other while you cheer her on. You can do this quickly, just take a few minutes to encourage her as she walks.
 
 2. **Drink Practice:**
    - Give Emma a cup without a lid and let her practice drinking water. Supervise her closely to enjoy the experience, and it’s perfectly fine for her to spill a little!
 
 These activities are quick but effective in promoting her gross motor skills while fitting into a busy schedule. Enjoy your time together!"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=False, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=Ge

In [14]:
await ask_gross_motor(
    "Please make the plan gentler; Emma currently cannot walk independently at all."
)


 ### Continue session: emma-gross-001

User > Please make the plan gentler; Emma currently cannot walk independently at all.
profile_agent_gross_motor > Here’s a gentler version of Day 3 activities tailored for Emma, focusing on supporting her as she works towards walking independently:

### Gentle Day 3 Activities:

1. **Assisted Walking Practice:**
   - Support Emma as she takes steps by holding her hands or encouraging her to use a sturdy piece of furniture for support. You can gently guide her while cheering her on, focusing on helping her feel confident.

2. **Cup Practice:**
   - Sit with Emma and let her practice holding a cup without a lid. You can help her guide the cup to her mouth and encourage her to drink. It's perfectly fine if there are spills; focus on the fun of trying!

These activities are designed to foster Emma's skills while ensuring she feels supported and safe. Enjoy this quality time together!
delay_agent_gross_motor > Here’s a gentler version of Day 3 activit

[Event(model_version='gpt-4o-mini-2024-07-18', content=Content(
   parts=[
     Part(
       text="""Here’s a gentler version of Day 3 activities tailored for Emma, focusing on supporting her as she works towards walking independently:
 
 ### Gentle Day 3 Activities:
 
 1. **Assisted Walking Practice:**
    - Support Emma as she takes steps by holding her hands or encouraging her to use a sturdy piece of furniture for support. You can gently guide her while cheering her on, focusing on helping her feel confident.
 
 2. **Cup Practice:**
    - Sit with Emma and let her practice holding a cup without a lid. You can help her guide the cup to her mouth and encourage her to drink. It's perfectly fine if there are spills; focus on the fun of trying!
 
 These activities are designed to foster Emma's skills while ensuring she feels supported and safe. Enjoy this quality time together!"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=False, turn_complete=None, finish_reason